In [1]:
pip install folium geopy

In [4]:
pip install ipyleaflet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00


In [10]:
pip install ipyleaflet


In [14]:
import folium

In [18]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=51f89f1600240bc2c90a68fb2027e5a550e845c0ffb938806173c68242c75b1d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [19]:
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [21]:
import wget
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [22]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:
# find coordinate of the closet coastline
#Lat =28.56367  Lon: -80.57163
coastline_lat = 28.59112
coastline_lon = -80.61999
launch_site_lat = 28.562302
launch_site_lon = -80.577356
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [27]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
coordinate = [launch_site_lat, launch_site_lon]
distance = distance_coastline
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

In [30]:
from ipyleaflet import Map, Marker, basemaps
from geopy.distance import geodesic
import folium

# Launch site coordinates (Cape Canaveral, Florida)
launch_site_coords = (28.562302, -80.577356)

# Coordinates of the closest city, coastline, railway, highway (example coordinates)
city_coords = (28.5383, -81.3792)  # Example coordinates for a city
coastline_coords = (28.392113, -80.576995)  # Example coordinates for coastline
railway_coords = (28.393207, -80.605469)  # Example coordinates for a railway
highway_coords = (28.399969, -80.605629)  # Example coordinates for a highway

# Create a map centered around the launch site
m = Map(center=launch_site_coords, zoom=10, basemap=basemaps.Esri.WorldImagery)

# Function to calculate distance between two points
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).miles

# Add markers for the closest city, coastline, railway, highway
city_marker = Marker(location=city_coords, draggable=False, title="City")
coastline_marker = Marker(location=coastline_coords, draggable=False, title="Coastline")
railway_marker = Marker(location=railway_coords, draggable=False, title="Railway")
highway_marker = Marker(location=highway_coords, draggable=False, title="Highway")
m.add_layer(city_marker)
m.add_layer(coastline_marker)
m.add_layer(railway_marker)
m.add_layer(highway_marker)

# Calculate distances to each point
city_distance = calculate_distance(launch_site_coords, city_coords)
coastline_distance = calculate_distance(launch_site_coords, coastline_coords)
railway_distance = calculate_distance(launch_site_coords, railway_coords)
highway_distance = calculate_distance(launch_site_coords, highway_coords)

# Print distances
print(f"Distance to closest city: {city_distance:.2f} miles")
print(f"Distance to closest coastline: {coastline_distance:.2f} miles")
print(f"Distance to closest railway: {railway_distance:.2f} miles")
print(f"Distance to closest highway: {highway_distance:.2f} miles")

# Create a folium map to draw lines between markers
folium_map = folium.Map(location=launch_site_coords, zoom_start=10)
folium.Marker(launch_site_coords, popup="Launch Site").add_to(folium_map)
folium.Marker(city_coords, popup="City").add_to(folium_map)
folium.Marker(coastline_coords, popup="Coastline").add_to(folium_map)
folium.Marker(railway_coords, popup="Railway").add_to(folium_map)
folium.Marker(highway_coords, popup="Highway").add_to(folium_map)
folium.PolyLine([launch_site_coords, city_coords], color="blue").add_to(folium_map)
folium.PolyLine([launch_site_coords, coastline_coords], color="red").add_to(folium_map)
folium.PolyLine([launch_site_coords, railway_coords], color="green").add_to(folium_map)
folium.PolyLine([launch_site_coords, highway_coords], color="orange").add_to(folium_map)

# Save the folium map as an HTML file
folium_map.save("map_with_distances.html")

# Display the ipyleaflet map
m

Distance to closest city: 48.78 miles
Distance to closest coastline: 11.72 miles
Distance to closest railway: 11.77 miles
Distance to closest highway: 11.31 miles


Map(center=[28.562302, -80.577356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…